In [1]:
1 :: (2 :: (3 :: []))

- : int list = [1; 2; 3]


In [2]:
1 :: 2 :: 3:: [];;

- : int list = [1; 2; 3]


可以看到，`::`是右结合的，这说明我们可以不用右括号来建立列表。空表`[]`用来结束一个列表。需要说明，空表是多态的，这意味者它可以用于任意类型的元素，如下所示

In [3]:
let empty = [];;

val empty : 'a list = []


In [4]:
3::empty;;

- : int list = [3]


In [5]:
"three"::empty;;

- : string list = ["three"]


Ocaml列表实际上是单列表

从一个列表中过滤掉列表中等于特定值的元素

In [6]:
let rec drop_value l to_drop = 
match l with
| [] -> []
| hd :: tl ->
let new_tl = drop_value tl to_drop in
if hd = to_drop then new_tl else hd :: new_tl

val drop_value : 'a list -> 'a -> 'a list = <fun>


In [7]:
drop_value [1;2;3;2] 2;;

- : int list = [1; 3]
